# Detectando objetos em videos com YOLOv4

## Etapa 1 - Download do Darknet

In [1]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15038, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 15038 (delta 12), reused 20 (delta 9), pack-reused 15002
Receiving objects: 100% (15038/15038), 13.41 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (10210/10210), done.


In [2]:
%cd darknet

/content/darknet


In [4]:
ls

3rdparty/               darknet_video.py        net_cam_v4.sh*
build/                  data/                   README.md
build.ps1*              image_yolov3.sh*        results/
cfg/                    image_yolov4.sh*        scripts/
cmake/                  include/                src/
CMakeLists.txt          json_mjpeg_streams.sh*  vcpkg.json
DarknetConfig.cmake.in  LICENSE                 video_yolov3.sh*
darknet_images.py       Makefile                video_yolov4.sh*
darknet.py              net_cam_v3.sh*


## Etapa 2 - Compilando a biblioteca

In [5]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [6]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

## Etapa 3 - Carregando os pesos do modelo pré-treinado

In [7]:
from google.colab import drive
drive.mount('/content/yolo')

Mounted at /content/yolo


In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [8]:
import zipfile
path = "/content/yolo/My Drive/yolo/modelo_YOLOv4.zip"
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

## Etapa 4 - Carregando o vídeo

### 4.1 - De uma url

In [10]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4

--2021-05-15 14:18:15--  https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_teste02.mp4 [following]
--2021-05-15 14:18:15--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_teste02.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7266878 (6.9M) [application/octet-stream]
Saving to: ‘video_teste02.mp4’

video_teste02.mp4   100%[===================>]   6.93M  26.9MB/s    in 0.3s    

2021-05-15 14:18:16 (26.9 MB/s) - ‘video_teste02.mp4’ saved [7266878/7266878]



### 4.2 - Do Google Drive


In [11]:
!cp /content/yolo/My\ Drive/yolo/videos/video_teste02.mp4 ./

In [12]:
ls

3rdparty/               darknet_video.py        README.md
backup/                 data/                   results/
build/                  image_yolov3.sh*        scripts/
build.ps1*              image_yolov4.sh*        src/
cfg/                    include/                vcpkg.json
cmake/                  json_mjpeg_streams.sh*  video_teste02.mp4
CMakeLists.txt          LICENSE                 video_yolov3.sh*
darknet*                Makefile                video_yolov4.sh*
DarknetConfig.cmake.in  net_cam_v3.sh*          yolov4.weights
darknet_images.py       net_cam_v4.sh*
darknet.py              obj/


## Etapa 5 - Realizando a detecção em vídeo

In [ ]:
!./darknet detect cfg/yolov4.cfg yolov4.weights data/person.jpg

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1/ 1    304 x 

In [13]:
# Aplicamos o detector que é o método similar ao detect que usamos anteriormente
# O parâmetro demo é obrigatório
# Passamos o coco.data que é a lista de classes, o yolov4.cfg e os pesos como de costume
# O padrão é ele ir apresentando enquanto faz a detecção, o -dont_show inibe essa apresentação durante a detecção
# o -i 0 indicam que o resultado será salvo em um arquivo externo
# O -out_filename dá o nome e caminho do arquivo
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_teste02.mp4 -i 0 -out_filename resultado.mp4 

#!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_teste02.mp4 -i 0 -out_filename resultado.avi

A saída de streaming foi truncada nas últimas 5000 linhas.
traffic light: 38% 
traffic light: 27% 
truck: 53% 
car: 61% 
truck: 31% 
car: 100% 
car: 100% 
car: 96% 
car: 94% 
car: 93% 
car: 93% 
car: 92% 
car: 85% 
car: 44% 

FPS:7.8 	 AVG_FPS:7.8

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 92% 
traffic light: 56% 
traffic light: 32% 
truck: 61% 
car: 53% 
truck: 31% 
car: 88% 
car: 100% 
car: 100% 
car: 97% 
car: 97% 
car: 94% 
car: 94% 
car: 86% 
car: 50% 
car: 28% 

FPS:7.8 	 AVG_FPS:7.8

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 93% 
traffic light: 64% 
traffic light: 44% 
truck: 58% 
car: 52% 
car: 100% 
car: 100% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 95% 
car: 91% 
car: 39% 

FPS:7.8 	 AVG_FPS:7.8

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 93% 
traffic light: 64% 
traffic light: 44% 
truck: 58% 
car: 51% 
car: 100% 
car: 100% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 95% 
car: 91% 
car: 39% 

FPS:7.8 	 AVG_FPS:7.8



In [15]:
# Exibe o arquivo
!du -h resultado.mp4

#!du -h resultado.avi

32M	resultado.mp4


## Etapa 6 - Visualizando o resultado

In [16]:
# Salvaremos no drive o modelo
!cp ./resultado.mp4 /content/yolo/My\ Drive/yolo/videos/resultado1.mp4

#!cp ./resultado.avi /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/videos/resultado1.avi

## Especificando um threshold

In [17]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4

--2021-05-15 14:23:36--  https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_rua01.mp4 [following]
--2021-05-15 14:23:37--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_rua01.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5645465 (5.4M) [application/octet-stream]
Saving to: ‘video_rua01.mp4’

video_rua01.mp4     100%[===================>]   5.38M  23.8MB/s    in 0.2s    

2021-05-15 14:23:38 (23.8 MB/s) - ‘video_rua01.mp4’ saved [5645465/5645465]



In [19]:
!cp /content/yolo/My\ Drive/yolo/videos/video_rua01.mp4 ./

In [23]:
# O atributo thresh de threshold indica um limiar para deteção
# Quanto maior, maior a probabilidade exigida para classificar um objeto

#!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_rua01.mp4 -out_filename resultado02.mp4 -thresh 0.01

!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_rua01.mp4 -out_filename resultado02.mp4 -thresh 0.03


#!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show video_rua01.mp4 -out_filename resultado02.avi -thresh 0.3

A saída de streaming foi truncada nas últimas 5000 linhas.
car: 29% 
truck: 2% 
car: 95% 
truck: 2% 
car: 85% 
truck: 1% 
person: 2% 
car: 99% 
car: 98% 
car: 72% 
car: 11% 
car: 6% 
car: 3% 
person: 4% , car: 2% 
person: 2% , car: 2% 
car: 2% 
person: 99% 
person: 98% 
person: 97% 
person: 95% 
person: 95% 
person: 94% 
person: 92% 
person: 91% 
person: 81% 
person: 74% 
person: 57% 
person: 54% 
person: 40% 
person: 30% 
person: 26% 
person: 16% 
person: 15% 
person: 14% 
person: 11% 
person: 10% 
person: 10% 
person: 8% 
person: 7% 
person: 5% 
person: 4% 
person: 4% 
person: 4% 
person: 3% 
person: 3% 
person: 3% 
person: 3% 
person: 3% 
person: 3% 
person: 3% 
person: 3% 
person: 2% 
person: 2% 
person: 2% 
person: 2% 
person: 2% 
person: 2% 
person: 2% 
person: 2% 
person: 2% 
person: 1% 
person: 1% 
person: 1% 
person: 1% 
person: 1% 
person: 1% 
person: 1% 
person: 1% 
person: 1% 

FPS:4.3 	 AVG_FPS:4.0

 cvWriteFrame 
Objects:

handbag: 53% , suitcase: 8% 
tie: 5% 
tie: 4% 
ha

In [24]:
!cp ./resultado02.mp4 /content/yolo/My\ Drive/yolo/videos/resultado2.mp4